In [5]:
import pandas as pd
import geopandas as gpd
from simple_modflow.modflow.mf6.voronoiplus import VoronoiGridPlus as Vor
from pathlib import Path
import shapely as shp
import pickle

class SFR:

    def __init__(
            self,
            vor: Vor = None,
            stream_path: Path = None
    ):
        self.vor = vor
        self.stream_path = stream_path
        self.stream_gdf = gpd.read_file(stream_path)
        self.stream_geom = self.stream_gdf.geometry
        self.stream_points = {}
        self.vor_intersect = {}

        for i, geom in enumerate(self.stream_geom):
            x, y = geom.coords.xy
            self.stream_points[i] = [shp.Point(point) for point in list(zip(x,y))]
            self.vor_intersect[i] = vor.get_vor_cells_as_series(geom)

In [1]:
from sfr import SFR
from pathlib import Path
import pickle
from simple_modflow.modflow.mf6.voronoiplus import VoronoiGridPlus as Vor

In [2]:
vor_path = Path(r'C:\Users\lukem\Python\MODFLOW\LakePointe\new_vor_lakepointe.vor')
with open(vor_path, 'rb') as file:
    vor: Vor = pickle.load(file)

In [3]:
input_path = Path(r'C:\Users\lukem\Python\MODFLOW\LakePointe\inputs')
stream = input_path / r'shp\rivers and streams\streams_simplified.shp'
sfr = SFR(vor=vor, stream_path=stream)

In [4]:
streams_cells = pd.concat(objs=[sfr.vor_intersect[i] for i in range(len(sfr.vor_intersect))]).to_list()
vor.show_selected_cells(streams_cells)

C:\Users\lukem\AppData\Local\Programs\Python\Python311\Lib\site-packages\nbformat\json_compat.py:14: DeprecationWarning:

Importing ErrorTree directly from the jsonschema package is deprecated and will become an ImportError. Import it from jsonschema.exceptions instead.

C:\Users\lukem\AppData\Local\Programs\Python\Python311\Lib\site-packages\nbformat\json_compat.py:14: DeprecationWarning:

Importing ErrorTree directly from the jsonschema package is deprecated and will become an ImportError. Import it from jsonschema.exceptions instead.



In [48]:
jenkins = sfr.stream_geom[1]
intersecting_cells = sfr.vor.gdf_vorPolys[ sfr.vor.gdf_vorPolys.intersects(jenkins)].copy()
# Calculate the centroid of each intersecting cell
intersecting_cells['centroid'] = intersecting_cells.centroid

# Calculate the distance along the stream line for each cell centroid
intersecting_cells['distance_along_stream'] = intersecting_cells.centroid.apply(
    lambda point: jenkins.project(point)
)
# Sort the cells based on the distance along the stream line
sorted_cells = intersecting_cells.sort_values('distance_along_stream')
sorted_cell_ids = sorted_cells.index.tolist()

In [50]:
sorted_cell_ids[::-1]

[1479,
 1467,
 104,
 1425,
 1290,
 1444,
 262,
 981,
 453,
 1085,
 1581,
 1219,
 560,
 210,
 907,
 354,
 521,
 10566,
 10565,
 10562,
 10245,
 10247,
 302,
 10244,
 10239,
 10242,
 555,
 10235,
 350,
 1686,
 1549,
 1654,
 209,
 1661,
 1649,
 1650,
 511,
 10331,
 10333,
 430,
 10335,
 10334,
 10537,
 208,
 10522,
 10506,
 10513,
 301,
 10503,
 554,
 10492,
 207,
 10486,
 10477,
 300,
 10463,
 10464,
 206,
 10457,
 10229,
 10292,
 387,
 10283,
 480,
 10304,
 205,
 10307,
 204,
 10316,
 10312,
 10636,
 203,
 10622,
 10588,
 10614,
 10600,
 202,
 10607,
 10608,
 540,
 10400,
 386,
 10397,
 10394,
 478,
 10382,
 201,
 10351,
 1531,
 10345,
 200,
 10355,
 428,
 10364,
 199,
 9973,
 198,
 9952,
 9956,
 299,
 9947,
 9942,
 9954,
 197,
 9967,
 196,
 9931,
 195,
 9926,
 9922,
 194,
 1621,
 10038,
 193,
 9908,
 192,
 9904,
 9895,
 9896,
 191,
 9885,
 385,
 9878,
 190,
 1518,
 427,
 1308,
 189,
 1300,
 467,
 1555,
 384,
 1562,
 1560,
 10185,
 188,
 10179,
 187,
 10184,
 390,
 10205,
 186,
 10203,


In [20]:
import geopandas as gpd
vor_idxs = sfr.vor_intersect_flat_list
reach_lens = []
for idx in vor_idxs:
    reach_len = sfr.stream_geom.unary_union.intersection(vor.gdf_vorPolys.loc[idx]).geometry.length
    reach_lens.append(reach_len)
reach_lens
    

[4.0000000000655564,
 35.004338221782966,
 8.984816507548228,
 13.876709521224448,
 31.012346515249824,
 15.9999999999732,
 16.000000000147924,
 12.006553611330006,
 26.917065013251815,
 19.138129514303817,
 29.630128563706528,
 9.956525792439564,
 20.662673611167666,
 14.66467180891197,
 32.000000000016264,
 22.276259029006912,
 29.630128563725,
 9.956525792665762,
 9.956525792472219,
 10.664671808992106,
 19.753419042522015,
 9.95652579256899,
 9.956525792665762,
 9.956525792665762,
 9.956525792472219,
 9.95652579256899,
 9.95652579256899,
 9.956525792665762,
 9.956525792439564,
 9.95652579256899,
 10.934788688702593,
 9.95652579256899,
 9.95652579256899,
 9.956525792342793,
 9.956525792665762,
 9.95652579256899,
 9.956525792665762,
 9.95652579256899,
 9.95652579256899,
 9.95652579256899,
 9.956525792439564,
 9.956525792665762,
 9.95652579256899,
 9.95652579256899,
 9.95652579256899,
 9.95652579256899,
 9.95652579256899,
 9.95652579256899,
 9.95652579256899,
 38.34166461096565,
 24.0

In [13]:
vor.gdf_vorPolys.loc[idx].array.sindex.query(sfr.stream_geom.unary_union, predicate='intersects')

AttributeError: 'GeoDataFrame' object has no attribute 'array'

In [18]:
sfr.stream_geom.unary_union.intersection(vor.gdf_vorPolys.loc[10868]).geometry.length

131.4763135465052

In [53]:
vor.gdf_vorPolys.loc[92]

geometry    POLYGON ((1243725.934 742939.512, 1243733.619 ...
Name: 92, dtype: geometry